In [45]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
from os import listdir
from os.path import isfile
import math

from datetime import datetime
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [46]:
class SequenceGenerator(keras.utils.Sequence):
    """
    A keras Sequence to be used as an image generator for the model.
    """

    def __init__(self, x, y, batchsize):
        self.x, self.y, self.batchsize = x, y, batchsize

    def __len__(self):
        return math.ceil(len(self.x) / self.batchsize)

    def names_at_batch(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])
        return x_names, y_names

    def __getitem__(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])

        # open x image names, resize, normalise and make a numpy array
        x1 = np.asarray([preprocess_input(img_to_array(load_img(file_name[0], target_size=(299, 299)))) for file_name in x_names])
        x2 = np.asarray([preprocess_input(img_to_array(load_img(file_name[1], target_size=(299, 299)))) for file_name in x_names])
        x3 = np.asarray([preprocess_input(img_to_array(load_img(file_name[2], target_size=(299, 299)))) for file_name in x_names])
        # x1 = np.asarray([img_to_array(load_img(file_name[0], target_size=(299, 299))) for file_name in x_names]) / 255.0
        # x2 = np.asarray([img_to_array(load_img(file_name[1], target_size=(299, 299))) for file_name in x_names]) / 255.0
        # x3 = np.asarray([img_to_array(load_img(file_name[2], target_size=(299, 299))) for file_name in x_names]) / 255.0

        return [x1, x2, x3], y_names

    def num_classes(self):
        ret = []
        for cat in self.y:
            if cat not in ret:
                ret.append(cat)
        return len(ret)

In [47]:
def shifted_arr(arr, shift):
    new_arr = [None, None, None]
    for i in range(3):
        new_arr[i] = arr[(i + shift) % 3][0]
    
    return new_arr

In [48]:
def get_filenames_from_dir(directory):
    x = []
    y = []
    category_count = 0
    for category in listdir(directory):
        triplet = []
        if isfile(category):
            continue
        for file in listdir("{0}/{1}".format(directory, category)):
            if file[-3:] != "jpg":
                continue
            triplet.append(("{0}/{1}/{2}".format(directory, category, file), category_count))
            if len(triplet) == 3:
                for i in range(3):
                    x.append(shifted_arr(triplet, i))
                    y.append(triplet[0][1])
                triplet = []
        category_count += 1

    return x, y

In [49]:
# makes arrays of the images and label names
x_names, y_names = get_filenames_from_dir("database")

# 15% of all the images are set aside as the test set
x_train_val, x_test, y_train_val, y_test = train_test_split(x_names, y_names, test_size=0.15, random_state=42)

# 17% of the non-test images are set aside as the validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.17, random_state=42)

In [50]:
# make generators with batch size 32 for each set
train_gen = SequenceGenerator(x_train, y_train, 4)
val_gen = SequenceGenerator(x_val, y_val, 4)
test_gen = SequenceGenerator(x_test, y_test, 4)

In [51]:
# Model 1
base_model1 = InceptionV3(weights="imagenet")
for layer in base_model1.layers:
    layer._name += "_1"
    layer.trainable = True

# Model 2
base_model2 = InceptionV3(weights="imagenet")
for layer in base_model2.layers:
    layer._name += "_2"
    layer.trainable = True

# Model 3
base_model3 = InceptionV3(weights="imagenet")
for layer in base_model3.layers:
    layer._name += "_3"
    layer.trainable = True

In [52]:
out_layer = keras.layers.concatenate([base_model1.layers[-2].output, base_model2.layers[-2].output, base_model3.layers[-2].output])
predictions = keras.layers.Dense(291, activation='softmax')(out_layer)

model = keras.Model(inputs=[base_model1.input, base_model2.input, base_model3.input], outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [53]:
logdir = "logs/3model_ensemb_shared_concat" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [54]:
model.fit(train_gen, validation_data=val_gen, callbacks=[tensorboard_callback], epochs=20)

Epoch 1/20
11126/11126 [==============================] - 2465s 219ms/step - loss: 4.9040 - accuracy: 0.0421 - val_loss: 23.1668 - val_accuracy: 0.0449
Epoch 2/20
11126/11126 [==============================] - 2391s 215ms/step - loss: 3.3397 - accuracy: 0.1981 - val_loss: 2.5583 - val_accuracy: 0.3140
Epoch 3/20
11126/11126 [==============================] - 2356s 212ms/step - loss: 1.9375 - accuracy: 0.4525 - val_loss: 1.6437 - val_accuracy: 0.5166
Epoch 4/20
11126/11126 [==============================] - 2368s 213ms/step - loss: 1.3185 - accuracy: 0.6005 - val_loss: 1.4488 - val_accuracy: 0.5744
Epoch 5/20
11126/11126 [==============================] - 2353s 211ms/step - loss: 0.9811 - accuracy: 0.6932 - val_loss: 1.2140 - val_accuracy: 0.6567
Epoch 6/20
11126/11126 [==============================] - 2371s 213ms/step - loss: 0.7695 - accuracy: 0.7557 - val_loss: 0.9501 - val_accuracy: 0.7262
Epoch 7/20
11126/11126 [==============================] - 2366s 213ms/step - loss: 0.6245 - a

In [55]:
model.evaluate(test_gen)

2366/2366 [==============================] - 154s 65ms/step - loss: 0.8073 - accuracy: 0.8217


[0.8073202967643738, 0.8217079043388367]